In [4]:
%load_ext Cython

import pandas as pd
import numpy as np

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [6]:
N = 1000000
posThreshold = 5
negThreshold = 7.5

index = pd.date_range('2000-01-01', periods=N, freq='min')
seriesData  = pd.Series(np.random.randn(N), name='data', index= index)

### Original Pandas Implementation 

Use Pandas + For Loop + Lists

In [5]:
def get_CUSUM_events_v2(inseries: pd.Series, posThreshold, negThreshold = None):
    """
    Implementation of the CUSUM filter, where E_{t − 1}[y_t] = y_{t − 1}
    :param inseries: the raw time series we wish to filter
    :param posThreshold:  the threshold for positive deltas in CUSUM
    :param negThreshold:  the threshold for negative deltas in CUSUM; if None, then posThreshold is used
    :return: a tuple (posEventIndex, negEventIndex) with subset of inseries.index elements,
            for each position where:
                posEventIndex: CUSUM-value was over the positive threshold (i.e. triggered an "alarm")
                negEventIndex: CUSUM-value was over the negative threshold
    Time needed:  38.22599673271179
    """

    negThreshold = negThreshold if negThreshold is not None else posThreshold

    posEventsList, negEventsList = [], []
    posSum, negSum = 0.0, 0.0
    diff = inseries.diff()
    for i in range(1, len(diff)):
        posSum = max(0, posSum + diff.iloc[i])
        negSum = min(0, negSum + diff.iloc[i])

        if negSum < -negThreshold:
            negSum = 0; negEventsList.append(i)
        elif posSum > posThreshold:
            posSum = 0; posEventsList.append(i)

    # Subset the original index of the series, and return them
    negEventIndex  = inseries.index[negEventsList]
    posEventIndex  = inseries.index[posEventsList]
    return posEventIndex, negEventIndex

In [7]:
%timeit get_CUSUM_events_v2(seriesData, posThreshold, negThreshold)

47.8 s ± 6.48 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### First Cython Implementation

Simply adding the data types

In [18]:
%%cython
cimport cython
import pandas as pd
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cpdef get_CUSUM_events_vc01(inseries: pd.Series, posThreshold, negThreshold = None):
    """
    Transcoded get_CUSUM_events_v2 to cython. 
    Cytonization level 1. Changed:
        * Changed syntax of declaration + initialization from a, b = v1, v2 (Python) to cdef <type> a = v1, <type> b = v2
        * Introduced C-vars for all loop vars
        * Replaced Python parameters by C vars since these are used in the loop (i.e. posThreshold_c, negThreshold_c)
        * Disabled some overflow, bounds, etc. checks
    Remain: access diff.iloc[i] uses python -> use view.
    Time needed:  15.864001989364624
    """
    negThreshold = negThreshold if negThreshold is not None else posThreshold

    posEventsList, negEventsList = [], []
    cdef double posSum = 0.0, negSum = 0.0
    cdef double value, posThreshold_c, negThreshold_c

    # Use C variables to speed up comparizon
    posThreshold_c = posThreshold
    negThreshold_c = negThreshold

    diff = inseries.diff()
    for i in range(1, len(diff)):
        value = diff.iloc[i]
        posSum = max(0, posSum + value)
        negSum = min(0, negSum + value)

        if negSum < -negThreshold_c:
            negSum = 0; negEventsList.append(i)
        elif posSum > posThreshold_c:
            posSum = 0; posEventsList.append(i)

    # Subset the original index of the series, and return them
    negEventIndex  = inseries.index[negEventsList]
    posEventIndex  = inseries.index[posEventsList]
    return posEventIndex, negEventIndex

In [19]:
%timeit get_CUSUM_events_vc01(seriesData, posThreshold, negThreshold)

22.8 s ± 3.24 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Cythonized Version


Use of memory-view

In [13]:
%%cython
cimport cython
import pandas as pd
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cpdef get_CUSUM_events_vc02(inseries: pd.Series, posThreshold, negThreshold = None):
    """
    Transcoded get_CUSUM_events_v2 to cython. 
    Cytonization level 2, from get_CUSUM_events_vc01. Changed:
        * all from get_CUSUM_events_vc01
        * Changed access diff.iloc[i] from pandas call to memoryview
        Time needed:  0.018997907638549805
        HUGE speedup by using memory view: from 15.8 sec to 0.019 sec (830x speedup)
    Next step: replace lists negEventsList and posEventsList by a C++ vector or such, and check possible improvement
    """

    negThreshold = negThreshold if negThreshold is not None else posThreshold

    posEventsList, negEventsList = [], []
    cdef double posSum = 0.0, negSum = 0.0
    cdef double value, posThreshold_c, negThreshold_c

    # Use C variables to speed up comparizon
    posThreshold_c = posThreshold
    negThreshold_c = negThreshold

    diff = inseries.diff()
    # introduce memview, and use pandas df.values (!), i.e. numpy array instead of pd dataframe
    cdef double [:] diff_memview = diff.values

    for i in range(1, len(diff)):
        # value = diff.iloc[i]  # original access
        value = diff_memview[i] # access via memory view
        posSum = max(0, posSum + value)
        negSum = min(0, negSum + value)

        if negSum < -negThreshold_c:
            negSum = 0; negEventsList.append(i)
        elif posSum > posThreshold_c:
            posSum = 0; posEventsList.append(i)

    # Subset the original index of the series, and return them
    negEventIndex  = inseries.index[negEventsList]
    posEventIndex  = inseries.index[posEventsList]
    return posEventIndex, negEventIndex



In [14]:
%timeit get_CUSUM_events_vc02(seriesData, posThreshold, negThreshold)

23 ms ± 387 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Numba Version

Using ND-Arrays instead of Pandas Series

In [42]:
import numba

@numba.jit
def get_CUSUM_events_numbav2(inseries: pd.Series, posThreshold, negThreshold = None):

    negThreshold = negThreshold if negThreshold is not None else posThreshold

    posEventsList, negEventsList = [], []
    posSum, negSum = 0.0, 0.0
    diff = inseries.diff().values
    for i in range(1, len(diff)):
        posSum = max(0, posSum + diff[i])
        negSum = min(0, negSum + diff[i])

        if negSum < -negThreshold:
            negSum = 0; negEventsList.append(i)
        elif posSum > posThreshold:
            posSum = 0; posEventsList.append(i)

    # Subset the original index of the series, and return them
    negEventIndex  = inseries.index[negEventsList]
    posEventIndex  = inseries.index[posEventsList]
    return posEventIndex, negEventIndex

In [47]:
%timeit get_CUSUM_events_numbav2(seriesData, posThreshold, negThreshold)

49.2 s ± 10.7 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
